A short notebook where I explore the concept of permutation importance using 
the [**ELI5 library**](https://eli5.readthedocs.io/en/latest/blackbox/permutation_importance.html). 

Inspired from this great tutorial: https://www.kaggle.com/dansbecker/permutation-importance. 

Enjoy!

In [2]:
import eli5
from eli5.sklearn import PermutationImportance
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
# Need this to display each ELI5 HTML report within the for loop.
from IPython.display import display
%matplotlib inline

In [3]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [4]:
SEED = 314
CV = KFold(n_splits=5)
FEATURES = train_df.drop(["target", "ID_code"], axis=1).columns.tolist()
TARGET_COL = "target"

In [5]:
for fold, (train_idx, valid_idx) in enumerate(CV.split(train_df, train_df[TARGET_COL])):
    clf = LGBMClassifier(random_state=SEED, n_threads=-1, 
                         eval_metric="auc", n_estimators=10000)
    clf.fit(train_df.loc[train_idx, FEATURES], 
            train_df.loc[train_idx, TARGET_COL], 
            eval_metric="auc",
            verbose=0,
            early_stopping_rounds=1000,
            eval_set=[(train_df.loc[valid_idx, FEATURES], 
                       train_df.loc[valid_idx, TARGET_COL])])
    permutation_importance = PermutationImportance(clf, random_state=SEED)
    permutation_importance.fit(train_df.loc[valid_idx, FEATURES], 
                               train_df.loc[valid_idx, TARGET_COL])
    print(f"Permutation importance for fold {fold}")
    display(eli5.show_weights(permutation_importance, feature_names = FEATURES))

Permutation importance for fold 0


Weight,Feature
0.0017 ± 0.0003,var_81
0.0014 ± 0.0006,var_174
0.0012 ± 0.0003,var_198
0.0011 ± 0.0006,var_139
0.0010 ± 0.0004,var_12
0.0009 ± 0.0004,var_121
0.0009 ± 0.0003,var_6
0.0009 ± 0.0005,var_13
0.0009 ± 0.0005,var_184
0.0008 ± 0.0005,var_40


Permutation importance for fold 1


Weight,Feature
0.0024 ± 0.0007,var_81
0.0022 ± 0.0003,var_12
0.0019 ± 0.0007,var_139
0.0018 ± 0.0003,var_80
0.0014 ± 0.0002,var_21
0.0013 ± 0.0003,var_26
0.0013 ± 0.0003,var_179
0.0012 ± 0.0004,var_53
0.0012 ± 0.0007,var_146
0.0011 ± 0.0003,var_174


Permutation importance for fold 2


Weight,Feature
0.0018 ± 0.0005,var_81
0.0014 ± 0.0002,var_76
0.0012 ± 0.0004,var_21
0.0010 ± 0.0006,var_26
0.0010 ± 0.0005,var_53
0.0010 ± 0.0004,var_0
0.0010 ± 0.0005,var_109
0.0010 ± 0.0004,var_165
0.0010 ± 0.0005,var_166
0.0009 ± 0.0005,var_110


Permutation importance for fold 3


Weight,Feature
0.0022 ± 0.0008,var_81
0.0010 ± 0.0004,var_12
0.0010 ± 0.0005,var_0
0.0009 ± 0.0006,var_148
0.0008 ± 0.0004,var_22
0.0008 ± 0.0005,var_53
0.0007 ± 0.0006,var_139
0.0007 ± 0.0003,var_198
0.0007 ± 0.0004,var_21
0.0007 ± 0.0004,var_184


Permutation importance for fold 4


Weight,Feature
0.0024 ± 0.0002,var_81
0.0016 ± 0.0007,var_21
0.0016 ± 0.0006,var_76
0.0014 ± 0.0003,var_6
0.0014 ± 0.0003,var_190
0.0013 ± 0.0002,var_2
0.0013 ± 0.0008,var_139
0.0013 ± 0.0006,var_146
0.0013 ± 0.0006,var_26
0.0013 ± 0.0004,var_44
